In [1]:
from aocd.models import Puzzle

def load_data(mode: str):
    if mode == "test":
        file = open("test.txt", "r")
        data = file.readlines()
        file.close()
    elif mode == 'input':
        data = Puzzle(2022, 7).input_data.splitlines()
    else:
        raise ValueError("This mode is not valid.")

    commands = []
    for line in data:
        commands.append(line.rstrip('\n'))
    return commands

In [2]:
from pathlib import Path
from collections import defaultdict

def compute_dir_size(commands: list[str]):
    tree = build_tree(commands)
    return sum([v for v in calculate_size(tree, defaultdict(int)).values() if v <= 100000])

def calculate_size(tree, path_to_size: defaultdict[str, int], dir_path=""):
    parent_folder = dir_path if dir_path else "/"
    for k, v in tree.items():
        if isinstance(v, dict):
            calculate_size(v, path_to_size, f"{dir_path}/{k}")
            path_to_size[parent_folder] += path_to_size[f"{dir_path}/{k}"]
        else:
            path_to_size[parent_folder] += v
    return path_to_size

def build_tree(commands: list[str]):
    tree = init_tree()
    curr = Path("/")
    for line in commands:
        cmd = line.split()
        if cmd[0] == "$":
            # command
            if cmd[1] == "cd":
                if cmd[2] == "/":
                    curr = Path("/")
                elif cmd[2] == "..":
                    curr = curr.parent
                else:
                    curr = curr / cmd[2]
            elif cmd[1] == "ls":
                pass
            else:
                print("ERROR")
        else:
            # not a command
            path = [p for p in str(curr).split("/") if p]
            target_dir = tree
            for p in path:
                target_dir = target_dir[p]
            if cmd[0] == "dir":
                target_dir[cmd[1]] = defaultdict()
            else:
                target_dir[cmd[1]] = int(cmd[0])
    return tree

def init_tree():
    return defaultdict(init_tree)

In [3]:
cmds = load_data("input")
answer = compute_dir_size(cmds)
print(answer)

1886043


In [4]:
def compute_dirs_to_remove(input_file: Path):
    max_size = 70000000
    update_size = 30000000
    
    tree = build_tree(input_file)
    path_to_sizes = calculate_size(tree, defaultdict(int))
    
    total_unused_space = max_size - path_to_sizes["/"]
    min_to_delete = update_size - total_unused_space
    
    assert min_to_delete > 0

    for dir_size in sorted(path_to_sizes.values()):
        if dir_size > min_to_delete:
            return dir_size
    raise RuntimeError("No way to free up enough space, we are doomed")

In [5]:
cmds = load_data("input")
answer = compute_dirs_to_remove(cmds)
print(answer)

3842121
